# Importing Libraries

In [1]:
# to ignore all the unnecessary warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
import cv2
#from coffeeshop.coffeeshop import Coffeeshop
from tensordash.tensordash import Tensordash
import matplotlib.pyplot as plt
import numpy as np
import glob
import keras
from keras.models import Sequential,Model
from keras.optimizers import Adam
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


# Importing Enhanced Data

In [3]:
x=list()
y=list()
for i in range(1,1101):
    img=cv2.imread("Fire 1/%d.jpg"%i,1)
    img=cv2.resize(img,(50,50))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(50, 50, 3)
    img=img.tolist()
    x.append(img)
    y.append(1)
for i in range(1,1101):
    img=cv2.imread("No Fire 1/%d.jpg"%i,1)
    img=cv2.resize(img,(50,50))
    img=np.float32(img)
    img/=255.0
    img = img.reshape(50, 50, 3)
    img=img.tolist()
    x.append(img)
    y.append(0)
x=np.asarray(x)
y=np.asarray(y)
y=to_categorical(y)

# Neural Network

In [4]:
def NN():
    model = Sequential()
    model.add(Conv2D(16, (3,3),input_shape=(50,50,3),padding='same', activation='sigmoid'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(8, (3, 3), activation = 'sigmoid',padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation = 'sigmoid'))
    model.add(Dense(64, activation = 'sigmoid'))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [5]:
model=NN()
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 16)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               147584    
_________________________________________________________________
dense_2 (Dense)              (None, 64)              

In [6]:
model.fit(x,y,epochs=20,batch_size=5,verbose=1,shuffle=True,validation_split=0.1)


Train on 1980 samples, validate on 220 samples
Epoch 1/20
1980/1980 [==============================] - 2s 1ms/step - loss: 0.6768 - accuracy: 0.5727 - val_loss: 0.6120 - val_accuracy: 0.7045
Epoch 2/20
1980/1980 [==============================] - 2s 1ms/step - loss: 0.4908 - accuracy: 0.7667 - val_loss: 0.6838 - val_accuracy: 0.6864
Epoch 3/20
1980/1980 [==============================] - 2s 1ms/step - loss: 0.4410 - accuracy: 0.8025 - val_loss: 0.7485 - val_accuracy: 0.6545
Epoch 4/20
1980/1980 [==============================] - 2s 1ms/step - loss: 0.4193 - accuracy: 0.8182 - val_loss: 0.6057 - val_accuracy: 0.7682
Epoch 5/20
1980/1980 [==============================] - 2s 1ms/step - loss: 0.4033 - accuracy: 0.8288 - val_loss: 1.1799 - val_accuracy: 0.4273
Epoch 6/20
1980/1980 [==============================] - 2s 1ms/step - loss: 0.4017 - accuracy: 0.8338 - val_loss: 0.6082 - val_accuracy: 0.7364
Epoch 7/20
1980/1980 [==============================] - 2s 1ms/step - loss: 0.3863 - acc

In [0]:
x

In [0]:
model.predict(x[76].reshape(1,50,50,3))

In [0]:
y

In [ ]:
model.save('val99.h5')